In [1]:
import pandas as pd
from config import BIN_SIZE as bin_size


In [2]:
# Load clinical data
clinical_df = pd.read_csv(
    "/labmed/workspace/lotta/finaletoolkit/dataframes_notebook/1-Tabelle 1.csv",
    sep=";",
    skiprows=1
)

expanded_rows = []

for _, row in clinical_df.iterrows():
    patient_field = str(row["Patient"])

    # Skip rows without "/"
    if "/" not in patient_field:
        continue

    # Split parts
    parts = patient_field.split("/")

    # Keep ONLY parts starting with "EE"
    ee_ids = [p for p in parts if p.startswith("EE")]
    print(ee_ids)

    # Skip rows with no EE ID
    if len(ee_ids) == 0:
        continue

    # Duplicate full row per EE ID
    for ee in ee_ids:
        new_row = row.copy()
        new_row["Extracted_ID"] = ee
        expanded_rows.append(new_row)

# Build final DataFrame
expanded_df = pd.DataFrame(expanded_rows)

# --- Reorder columns: Patient, Extracted_ID, then all others ---
cols = expanded_df.columns.tolist()
cols.remove("Extracted_ID")
cols.insert(1, "Extracted_ID")  # position 1 = second column
expanded_df = expanded_df[cols]

print(expanded_df.head())

# Save result
# Name der Spalte, die die Gruppe enthält
group_col = "Patient Type"   # ggf. anpassen

# Healthy nach oben holen: Healthy = 0, alle anderen = 1
expanded_df["sorting_key"] = expanded_df[group_col].apply(
    lambda x: 0 if str(x).lower() == "healthy" else 1
)

# Danach normal nach dem Gruppennamen sortieren
expanded_df = expanded_df.sort_values(
    by=["sorting_key", group_col]
).drop(columns=["sorting_key"])

expanded_df = expanded_df.drop_duplicates(subset=['Extracted_ID'], keep='first')

# Entferne doppelte IDs am Ende (falls gleiche Proben unter versch. Namen auftauchen)
initial_count = len(expanded_df)
expanded_df = expanded_df.drop_duplicates(subset=['Extracted_ID'], keep='first')
print(f'Bereinigung: {initial_count - len(expanded_df)} Duplikate basierend auf Extracted_ID entfernt.')

expanded_df.to_csv(
    "/labmed/workspace/lotta/finaletoolkit/dataframes_notebook/filtered_clinical_characteristics.csv",
    index=False, sep=";")


['EE87865']
['EE87866']
['EE87867']
['EE87868']
['EE87869']
['EE87870']
['EE87871']
['EE87872']
['EE87873']
['EE87874']
['EE87875']
['EE87876']
['EE87877']
['EE87878']
['EE87879']
['EE87880']
['EE87881']
['EE87882']
['EE87883']
['EE87884']
['EE87885']
['EE87886']
['EE87887']
['EE87888']
['EE87889']
['EE87890']
['EE87891']
['EE87922']
['EE87923']
['EE87924']
['EE87925']
['EE87926']
['EE87927']
['EE87928']
['EE87929']
['EE87931']
['EE87932']
['EE87933']
['EE87934']
['EE87935']
['EE87936']
['EE87937']
['EE87938']
['EE87939']
['EE87940']
['EE87941']
['EE87942']
['EE87943']
['EE87944']
['EE87945']
['EE87946']
['EE87947']
['EE87948']
['EE87949']
['EE87950']
['EE87951']
['EE87952']
['EE87953']
['EE87954']
['EE87955']
['EE87956']
['EE87957']
['EE87958']
['EE87959']
['EE87960']
['EE87961']
['EE87962']
['EE87963']
['EE87964']
['EE87965']
['EE87966']
['EE87967']
['EE87968']
['EE87969']
['EE87970']
['EE87971']
['EE87972']
['EE87973']
['EE87974']
['EE87975']
['EE87976']
['EE87977']
['EE87978']
['EE

In [3]:
print(expanded_df[expanded_df['Extracted_ID']=='EE88323'])

                      Patient Extracted_ID       Patient Type Sample Type  \
524  CGPLPA95/EE88323/EE88324      EE88323  Pancreatic Cancer       cfDNA   

                        Timepoint  Age at Diagnosis Gender Stage TNM Staging  \
524  Preoperative treatment naïve              64.0      F    II         NaN   

    Site of Primary Tumor Histopathological Diagnosis  \
524              Pancreas       Ductal Adenocarcinoma   

    Degree of Differentiation Location of Metastases at Diagnosis  \
524                      Well                          Lymph Node   

    Volume of Plasma (ml) cfDNA Extracted (ng/ml) cfDNA Input (ng/ml)  \
524                   NaN                     NaN                 NaN   

    Whole Genome Fragment Profile  Analysis  \
524                                       Y   

    Targeted Fragment Profile Analysis Targeted Mutation Analysis  
524                                  N                          N  


In [4]:
# Load GC corrected feature matrix
matrix_df = pd.read_csv(f"/labmed/workspace/lotta/finaletoolkit/dataframes_for_ba/final_feature_matrix_gc_corrected_{bin_size}.tsv", sep="\t")

# Get unique IDs from the matrix
matrix_ids = set(matrix_df['sample'].unique())
print(f"Number of unique samples in GC corrected matrix: {len(matrix_ids)}")

Number of unique samples in GC corrected matrix: 262


In [5]:
# Get the set of extracted clinical IDs
expanded_df = pd.read_csv("/labmed/workspace/lotta/finaletoolkit/dataframes_for_ba/filtered_clinical_characteristics.csv", sep=";")
clinical_ids = set(expanded_df['Extracted_ID'].unique())

# Calculate overlaps and differences
intersection = matrix_ids.intersection(clinical_ids)
matrix_only = matrix_ids - clinical_ids
clinical_only = clinical_ids - matrix_ids

print("--- Sample Count Analysis ---")
print(f"Samples in BOTH Matrix and Clinical table: {len(intersection)}")
print(f"Samples ONLY in Matrix (Missing from Clinical): {len(matrix_only)}")
print(f"Samples ONLY in Clinical (Missing from Matrix): {len(clinical_only)}")

print(f"\nTotal Unique Samples in Matrix: {len(matrix_ids)} (Should be {len(intersection)} + {len(matrix_only)}) ")
print(f"Total Unique Samples in Clinical: {len(clinical_ids)} (Should be {len(intersection)} + {len(clinical_only)})")

print("\n--- Details ---")
print("Samples in GC corrected matrix but NOT in the filtered clinical table:")
print(sorted(list(matrix_only)))

if clinical_only:
    print("\nSamples in Clinical table but NOT in the matrix (This explains the count discrepancy):")
    print(sorted(list(clinical_only)))

# Get cancer types (group) for missing IDs (Matrix Only)
if matrix_only:
    missing_info = matrix_df[matrix_df['sample'].isin(matrix_only)][['sample', 'group']].drop_duplicates()
    print("\nCancer types for samples missing from clinical table:")
    print(missing_info.to_string(index=False))


--- Sample Count Analysis ---
Samples in BOTH Matrix and Clinical table: 246
Samples ONLY in Matrix (Missing from Clinical): 16
Samples ONLY in Clinical (Missing from Matrix): 0

Total Unique Samples in Matrix: 262 (Should be 246 + 16) 
Total Unique Samples in Clinical: 246 (Should be 246 + 0)

--- Details ---
Samples in GC corrected matrix but NOT in the filtered clinical table:
['EE85727', 'EE85730', 'EE85731', 'EE85732', 'EE85733', 'EE85734', 'EE85737', 'EE85739', 'EE85741', 'EE85743', 'EE85746', 'EE85749', 'EE85750', 'EE85752', 'EE85753', 'EE88324']

Cancer types for samples missing from clinical table:
 sample      group
EE85727 colorectal
EE85730 colorectal
EE85731 colorectal
EE85732 colorectal
EE85733 colorectal
EE85734 colorectal
EE85737 colorectal
EE85739 colorectal
EE85741 colorectal
EE85743 colorectal
EE85746 colorectal
EE85749 colorectal
EE85750 colorectal
EE85752 colorectal
EE85753 colorectal
EE88324 pancreatic


TBR*** (Colorectal Cancer) fehlen in Clinical Matrix

SRR2130010 / IC11 EE86234 colorectal: male, Snyder 

SRR2130031 / IC33 / EE86255 colorectal: male, Snyder

SRR2130035 /IC37 /EE86259 colorectal: female, Snyder



SRR2130051 / IH02 / EE86275 healthy: male, Snyder

SRR2130052 / IH03 / EE86276 healthy: female, Snyder

CGPLH311 / EE87945    healthy, Cristiano

CGPLH335 / EE87965    healthy, Cristiano



IC47 / SRR2130044 / EE86268 pancreatic: female, Snyder

IC49 / SRR2130046 / EE86270 pancreatic: male, Snyder

IC50 / SRR2130047 / EE86271 pancreatic: female, Snyder

IC51 / SRR2130048 / EE86272 pancreatic: female, Snyder

IC52 / SRR2130049 / EE86273 pancreatic: male, Snyder



In [6]:
new_samples = [
    {"Patient": "SRR2130031/IC33/EE86255", "Extracted_ID": "EE86255", "Patient Type": "Colorectal Cancer", "Gender": "M"},
    {"Patient": "SRR2130035/IC37/EE86259", "Extracted_ID": "EE86259", "Patient Type": "Colorectal Cancer", "Gender": "F"},
    {"Patient": "SRR2130051/IH02/EE86275", "Extracted_ID": "EE86275", "Patient Type": "Healthy", "Gender": "M"},
    {"Patient": "SRR2130052/IH03/EE86276", "Extracted_ID": "EE86276", "Patient Type": "Healthy", "Gender": "F"},
    {"Patient": "CGPLH311/EE87945", "Extracted_ID": "EE87945", "Patient Type": "Healthy"},
    {"Patient": "CGPLH335/EE87965", "Extracted_ID": "EE87965", "Patient Type": "Healthy"},
    {"Patient": "IC47/SRR2130044/EE86268", "Extracted_ID": "EE86268", "Patient Type": "Pancreatic Cancer", "Gender": "F"},
    {"Patient": "IC49/SRR2130046/EE86270", "Extracted_ID": "EE86270", "Patient Type": "Pancreatic Cancer", "Gender": "M"},
    {"Patient": "IC50/SRR2130047/EE86271", "Extracted_ID": "EE86271", "Patient Type": "Pancreatic Cancer", "Gender": "F"},
    {"Patient": "IC51/SRR2130048/EE86272", "Extracted_ID": "EE86272", "Patient Type": "Pancreatic Cancer", "Gender": "F"},
    {"Patient": "IC52/SRR2130049/EE86273", "Extracted_ID": "EE86273", "Patient Type": "Pancreatic Cancer", "Gender": "M"},
    {"Patient": "SRR2130010/IC11/EE86234", "Extracted_ID": "EE86234", "Patient Type": "Colorectal Cancer", "Gender": "M"}
]
healthy_jiang_samples = [
    {"Patient": "C360/EE85898", "Extracted_ID": "EE85898", "Patient Type": "Healthy", "Gender": "M"},
    {"Patient": "C325/EE85904", "Extracted_ID": "EE85904", "Patient Type": "Healthy", "Gender": "M"},
    {"Patient": "C352/EE85905", "Extracted_ID": "EE85905", "Patient Type": "Healthy", "Gender": "M"},
    {"Patient": "C358/EE85908", "Extracted_ID": "EE85908", "Patient Type": "Healthy", "Gender": "F"},
    {"Patient": "C355/EE85918", "Extracted_ID": "EE85918", "Patient Type": "Healthy", "Gender": "F"},
    {"Patient": "C310/EE85928", "Extracted_ID": "EE85928", "Patient Type": "Healthy", "Gender": "F"},
    {"Patient": "C346/EE85936", "Extracted_ID": "EE85936", "Patient Type": "Healthy", "Gender": "F"},
    {"Patient": "C331/EE85937", "Extracted_ID": "EE85937", "Patient Type": "Healthy", "Gender": "M"},
    {"Patient": "C309/EE85941", "Extracted_ID": "EE85941", "Patient Type": "Healthy", "Gender": "F"},
    {"Patient": "C348/EE85959", "Extracted_ID": "EE85959", "Patient Type": "Healthy", "Gender": "M"},
    {"Patient": "C314/EE85963", "Extracted_ID": "EE85963", "Patient Type": "Healthy", "Gender": "M"},
    {"Patient": "C351/EE85970", "Extracted_ID": "EE85970", "Patient Type": "Healthy", "Gender": "M"},
    {"Patient": "C326/EE85971", "Extracted_ID": "EE85971", "Patient Type": "Healthy", "Gender": "M"},
    {"Patient": "C356/EE85980", "Extracted_ID": "EE85980", "Patient Type": "Healthy", "Gender": "F"},
    {"Patient": "C313/EE85985", "Extracted_ID": "EE85985", "Patient Type": "Healthy", "Gender": "M"},
    {"Patient": "C332/EE85987", "Extracted_ID": "EE85987", "Patient Type": "Healthy", "Gender": "F"},
    {"Patient": "C345/EE85988", "Extracted_ID": "EE85988", "Patient Type": "Healthy", "Gender": "F"},
]

import pandas as pd
existing_ids = set(expanded_df["Extracted_ID"].unique())

def add_unique_samples(df, new_list, existing_ids):
    added = 0
    rows_to_add = []
    for s in new_list:
        if s["Extracted_ID"] not in existing_ids:
            rows_to_add.append(s)
            existing_ids.add(s["Extracted_ID"])
            added += 1
    
    if rows_to_add:
        df = pd.concat([df, pd.DataFrame(rows_to_add)], ignore_index=True)
    return df, added

expanded_df, count1 = add_unique_samples(expanded_df, new_samples, existing_ids)
expanded_df, count2 = add_unique_samples(expanded_df, healthy_jiang_samples, existing_ids)

print(f'Added {count1 + count2} new unique samples. (Skipped those already present)')

expanded_df = expanded_df.drop_duplicates(subset=['Extracted_ID'], keep='first')

# Entferne doppelte IDs am Ende (falls gleiche Proben unter versch. Namen auftauchen)
initial_count = len(expanded_df)
expanded_df = expanded_df.drop_duplicates(subset=['Extracted_ID'], keep='first')
print(f'Bereinigung: {initial_count - len(expanded_df)} Duplikate basierend auf Extracted_ID entfernt.')

expanded_df.to_csv(
    '/labmed/workspace/lotta/finaletoolkit/dataframes_for_ba/filtered_clinical_characteristics.csv',
    index=False, sep=";")


Added 0 new unique samples. (Skipped those already present)
Bereinigung: 0 Duplikate basierend auf Extracted_ID entfernt.


In [7]:
healthy_count = len(expanded_df[expanded_df["Patient Type"].str.lower() == "healthy"])
cancer_count = len(expanded_df[expanded_df["Patient Type"].str.lower() != "healthy"])
print(f"Anzahl Healthy: {healthy_count}")
print(f"Anzahl Cancer: {cancer_count}")
print(expanded_df[expanded_df["Patient Type"].str.lower() == "healthy"]["Extracted_ID"])
# gebe die extratced ids aus die doppelt vorkommen 
# gebe die extratced id aus bei denen true bei duplicated erscheint 

print(expanded_df[expanded_df["Patient Type"].str.lower() == "healthy"]["Extracted_ID"].duplicated())=="True"
print(expanded_df[expanded_df["Patient Type"].str.lower() == "healthy"]["Extracted_ID"].duplicated().sum())



Anzahl Healthy: 131
Anzahl Cancer: 115
0      EE87922
1      EE87923
2      EE87924
3      EE87925
4      EE87926
        ...   
241    EE85971
242    EE85980
243    EE85985
244    EE85987
245    EE85988
Name: Extracted_ID, Length: 131, dtype: object
0      False
1      False
2      False
3      False
4      False
       ...  
241    False
242    False
243    False
244    False
245    False
Name: Extracted_ID, Length: 131, dtype: bool
0
